# 評価方法

分類されていないデータを認識し、どれだけ正しくカテゴリごとに分類できるかを算出した「平均精度」の高さを競い合います。

今回、活用するデータはLSWMD_25519となります。
LSWMD_25519のFailureType項目が分類されていない状態のデータに対し、正しいFailureTypeカテゴリを分類するプログラムを作成し、その平均精度を算出します。
平均精度とは、カテゴリごとに正しく分類できる精度を平均した値です。カテゴリごとに算出した精度（Aが正しく分類された数/Aのデータ数）を足し、カテゴリ数で割ります。

公平な評価を実施するために、以下の制限を設けています。
1. 外部パッケージをインストールするためのセルとsolution関数の中身のみを編集すること
2. 校舎のiMac上で最後のセルの実行時間が15分未満であること　（%%timeitの出力結果を確認してください）

※気になる点がある場合、Discordで気軽にお問合せください。

In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

外部パッケージを使用する場合、以下の方法でインストールを実施してください。

In [53]:
# 必要な外部パッケージは、以下の内容を編集しインストールしてください
# !pip install keras
# !pip install tensorflow
# !pip install opencv-python
import timeit
import tensorflow as tf
from keras import models
from keras import layers
from scipy.ndimage import zoom
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [54]:
# データの水増しをする関数
def augment_data(images, y_labels):
    augmented_images = []
    augmented_labels = []

    for img, label in zip(images, y_labels):
        # 元の画像
        augmented_images.append(img)
        augmented_labels.append(label)

        # 垂直方向の反転
        img_v_flip = np.flipud(img)
        augmented_images.append(img_v_flip)
        augmented_labels.append(label)

        # 水平方向の反転
        img_h_flip = np.fliplr(img)
        augmented_images.append(img_h_flip)
        augmented_labels.append(label)

        # 90度、180度、270度回転
        for angle in [90, 180, 270]:
            img_rotated = np.rot90(img, k=angle // 90)
            augmented_images.append(img_rotated)
            augmented_labels.append(label)

    return np.array(augmented_images), np.array(augmented_labels)

In [55]:
# 画像のサイズを整える関数
def resize_array(image_array, target_shape=(48, 48)):
    # Numpy配列からPillowのImageオブジェクトに変換
    image = Image.fromarray(image_array)
    # 画像を32x32ピクセルにリサイズ
    resized_image = image.resize(target_shape)
    # リサイズした画像をNumpy配列に戻す
    resized_image_array = np.array(resized_image)

    return resized_image_array

In [56]:
# ゲインフィルタのための関数
def apply_custom_filter(image_array, mask):
    # マスクに基づいて条件を満たすピクセル値を変更
    filtered_image_array = np.where(np.logical_and(mask, image_array == 2), 3, image_array)

    return filtered_image_array

def process_images(images, mask):
    # 各画像に対してフィルタを適用
    processed_images = [apply_custom_filter(image, mask) for image in images]

    return processed_images

def create_shifted_circular_mask(h, w, shift=(-1, -1), radius=None, edge_width=3):
    center = (int(w/2) + shift[0], int(h/2) + shift[1])
    # 半径が指定されていない場合、最大サイズを使用
    if radius is None:
        radius = min(center[0], center[1], w-center[0], h-center[1]) + 1

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    # 円の外側と内側の境界を作成
    outer_mask = dist_from_center <= radius
    inner_mask = dist_from_center <= (radius - edge_width)

    # 円のエッジ部分のみを取得
    edge_mask = np.logical_and(outer_mask, np.logical_not(inner_mask))
    return edge_mask


In [57]:
# Near-fullのためのフィルタ
def replace_values_in_images(images, target_value, replacement_value, threshold_ratio):
    processed_images = []

    for image in images:
        # 特定値のピクセル数をカウント
        target_pixel_count = np.sum(image == target_value)

        # 画像全体のピクセル数
        total_pixel_count = image.size

        # 存在比の計算
        ratio = target_pixel_count / total_pixel_count

        # 閾値以上であれば置き換え
        if ratio >= threshold_ratio:
            image[image == target_value] = replacement_value

        processed_images.append(image)

    return processed_images

In [58]:
def solution(x_test_df, train_df):
    # モデルのfailureTypeをIntに置き換える
    labels2 = ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']
    for i in range(len(labels2)):
        train_df.loc[train_df["failureType"]==labels2[i], "num"] = i

    
    print("===preprocessing start===")

    # 画像のサイズを全て揃える
    # 32x32になっているが、サイズは上のセルのresize_arrayから変更できる　今xは(要素数,32,32)の3次元Numpy配列になっている
    img = train_df["waferMap"]
    resized_arrays = [resize_array(arr) for arr in img]
    final_array = np.array(resized_arrays)

    # 画像にフィルタをかける
    # Near-fullフィルタ
    nearfull_threshold = 0.525
    nearfull_changedvalue = 255
    nearfull_filtered_images = replace_values_in_images(final_array, 2, nearfull_changedvalue, nearfull_threshold)

    # Edgeフィルタ
    # 円形マスクを作成
    circular_gain_mask = create_shifted_circular_mask(48, 48)
    # フィルタを再度適用
    filtered_images = process_images(nearfull_filtered_images, circular_gain_mask)

    # モデル訓練用データの答え（y）だけ取り出す
    y_train_df = train_df["num"]

    # モデル訓練用データを水増しする（画像を回したり反転させたりする。結果は変わらないので・・・）
    augmented_train_images, augmented_y_train = augment_data(filtered_images, y_train_df.values)

	# One-hotエンコーディング(推測したいfailureTypeが0-7の値になっているのを、7列のTrue or Falseに書き換える)
    ydf = pd.get_dummies(augmented_y_train)

    print("===preprocessing end===")
    
    # レイヤーを設定し学習
    model=models.Sequential()

    model.add(layers.Conv2D(16,(3,3), input_shape=(48,48,1), activation='relu'))
    model.add(layers.Conv2D(32,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(64,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))   
    
    model.add(layers.Flatten())

    model.add(layers.Dense(32, activation='gelu'))
    model.add(layers.Dense(32, activation='gelu'))
    #model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation='gelu'))
    model.add(layers.Dense(32, activation='gelu'))
    model.add(layers.Dense(16, activation='gelu'))
    model.add(layers.Dense(16, activation='gelu'))
    model.add(layers.Dense(16, activation='gelu'))
    model.add(layers.Dense(16, activation='gelu'))
    model.add(layers.Dense(8, activation='softmax'))
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    
    results = model.fit(augmented_train_images,
                      ydf,
                      epochs=5,
                      batch_size=60,
                      verbose=1,
                      validation_split=0.0)
    print("学習終了")

    # 試験用データ(x)の画像のサイズを学習データと揃える
    aimg = x_test_df["waferMap"]
    aresized_arrays = [resize_array(arr) for arr in aimg]
    anearfull_arrays = replace_values_in_images(aresized_arrays, 2, nearfull_changedvalue, nearfull_threshold)
    afiltered_arrays = process_images(anearfull_arrays, circular_gain_mask)
    afinal_array = np.array(afiltered_arrays)
    
    # 試験用データに回答する
    predictions = model.predict(afinal_array)

    # 出力された回答データ(y)はIntなので、failureTypeに直してReturnする
    res = np.zeros(predictions.shape[0], dtype=object)
    for i in range(predictions.shape[0]):
        res[i] = labels2[np.argmax(predictions[i])]

    return pd.DataFrame(res, index=x_test_df.index, columns=['failureType'])


solution関数は以下のように活用され、平均精度を計算します。

In [59]:
%%timeit -r 1 -n 1

# データのインポート
df=pd.read_pickle("../input/LSWMD_25519.pkl")

# テスト用と学習用のデータを作成（テストする際は、random_stateの値などを編集してみてください）
train_df, test_df = train_test_split(df, stratify=df['failureType'], test_size=0.10, random_state=2500)

y_test_df = test_df[['failureType']]
x_test_df = test_df.drop(columns=['failureType'])

# solution関数を実行
user_result_df = solution(x_test_df, train_df)

average_accuracy = 0
# ユーザーの提出物のフォーマット確認
if type(y_test_df) == type(user_result_df) and y_test_df.shape == user_result_df.shape:
    # 平均精度の計算
    accuracies = {}
    for failure_type in df['failureType'].unique():
        y_test_df_by_failure_type = y_test_df[y_test_df['failureType'] == failure_type]
        user_result_df_by_failure_type = user_result_df[y_test_df['failureType'] == failure_type]
        matching_rows = (y_test_df_by_failure_type == user_result_df_by_failure_type).all(axis=1).sum()
        accuracies[failure_type] = (matching_rows/(len(y_test_df_by_failure_type)))
    
    average_accuracy = sum(accuracies.values())/len(accuracies)

# #追加コード
# # テストデータとユーザーの提出物からランダムにサンプルを選択
# sample_size = 200

# # ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full'] から最低でも1つは選択する
# # クラスと対応する確率を設定
# selected_classes = ['Center', 'Donut', 'Edge-Loc', 'Edge-Ring', 'Loc', 'Random', 'Scratch', 'Near-full']
# # probabilities = [0.1 if class_name != 'Near-full' else 0.9 for class_name in selected_classes]
# probabilities = [0.1 if class_name != 'Near-full' else 0.9 for class_name in y_test_df['failureType']]

# # 確率の合計が1になるように正規化
# probabilities /= np.sum(probabilities)
# random_indices = np.random.choice(len(y_test_df), sample_size, replace=False, p=probabilities)

# # 実際のクラスと予測クラスを取得
# actual_classes = y_test_df.iloc[random_indices]['failureType'].values
# predicted_classes = user_result_df.iloc[random_indices]['failureType'].values
# # print("実際のクラス:", actual_classes)
# # print("予測クラス:", predicted_classes)
# # 混同行列を作成
# matrix = confusion_matrix(actual_classes, predicted_classes)
# labels3 = ['Center','Donut','Edge-Loc','Edge-Ring','Loc','Random','Scratch','Near-full']
# dataframe = pd.DataFrame(matrix, index=labels3, columns=labels3)
# #ヒートマップを作成
# sns.heatmap(dataframe, annot=True, cbar=None, cmap="Blues")
# plt.title("Confusion Matrix"), plt.tight_layout()
# plt.ylabel("True Class"), plt.xlabel("Predicted Class")
# plt.show()
# # コードはここで終わり

print(f"平均精度：{average_accuracy*100:.2f}%")

===preprocessing start===
===preprocessing end===
Epoch 1/5
2297/2297 [==============================] - 86s 37ms/step - loss: 0.6509 - accuracy: 0.7634
Epoch 2/5
2297/2297 [==============================] - 90s 39ms/step - loss: 0.3373 - accuracy: 0.8752
Epoch 3/5
2297/2297 [==============================] - 112s 49ms/step - loss: 0.2641 - accuracy: 0.9035
Epoch 4/5
2297/2297 [==============================] - 110s 48ms/step - loss: 0.2036 - accuracy: 0.9265
Epoch 5/5
2297/2297 [==============================] - 110s 48ms/step - loss: 0.1710 - accuracy: 0.9388
学習終了
80/80 [==============================] - 1s 8ms/step
平均精度：87.21%
8min 31s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
